In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.mlab import bivariate_normal
import pandas as pd
from dateutil.parser import parse
import numpy as np
import warnings
import datetime as dt
import seaborn as sns
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
vistas = pd.read_csv('../data/adri_data/entrenamiento_vistas.csv');
vistas.head()

,idaviso,idpostulante,month,day,dayofweek,hour,valor_educacion,sexo,edad,idnombre_area_pref,...,titulo,descripcion,contenido,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa
0,1112400775,kPjr2K0,4,6,4,5,9,1,20,182,...,576531138227470336,7670335039548230284,7597924801920901552,0,0,0,0,0,182,489
1,1112400775,kPjr2K0,4,6,4,5,9,1,20,182,...,576531138227470336,7670335039548230284,7597924801920901552,0,0,0,0,0,182,489
2,1112400775,kPjr2K0,4,6,4,5,9,1,20,182,...,576531138227470336,7670335039548230284,7597924801920901552,0,0,0,0,0,182,489
3,1112400775,rmRpqEA,4,4,2,2,12,1,33,2,...,576531138227470336,7670335039548230284,7597924801920901552,0,0,0,0,0,182,489
4,1112400775,EzEovD2,4,6,4,5,4,1,22,182,...,576531138227470336,7670335039548230284,7597924801920901552,0,0,0,0,0,182,489


In [3]:
postulaciones = pd.read_csv('../data/adri_data/entrenamiento_postulaciones.csv');
postulaciones.head()

,idaviso,esta_online,titulo,descripcion,contenido,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,...,day,dayofweek,hour,valor_educacion,sexo,edad,idnombre_area_pref,idnivel_laboral_pref,idtipo_de_trabajo_pref,idpostulante_int
0,1111556097,0,1120986480640,8230659354068575249,8159438729029635585,0,0,0,0,0,...,13,1,16,17,1,44,119,0,0,446521
1,1112370711,1,5629499534229504,4305833638884806680,4306765247469621377,0,0,0,0,0,...,8,3,14,17,1,44,119,0,0,446521
2,1112326655,1,16384,7886788350607107624,7959338346396781218,0,0,0,0,0,...,9,4,14,17,1,44,119,0,0,446521
3,1112350182,1,4611686018427404288,8861078463061660440,8789020298799968481,0,0,0,0,0,...,20,1,15,17,1,44,119,0,0,446521
4,1112383837,1,16392,6440571711296978984,6513419378897462402,0,0,0,0,0,...,7,2,14,17,1,44,119,0,0,446521


In [4]:
postulaciones.columns

Index(['idaviso', 'esta_online', 'titulo', 'descripcion', 'contenido',
       'idzona', 'idciudad', 'idmapacalle', 'idtipo_de_trabajo',
       'idnivel_laboral', 'idnombre_area', 'iddenominacion_empresa',
       'idpostulante', 'month', 'day', 'dayofweek', 'hour', 'valor_educacion',
       'sexo', 'edad', 'idnombre_area_pref', 'idnivel_laboral_pref',
       'idtipo_de_trabajo_pref', 'idpostulante_int'],
      dtype='object')

In [5]:
vistas = vistas[postulaciones.columns]

In [6]:
postulaciones = postulaciones[postulaciones.columns]

In [7]:
vistas['se_postulo'] = 0
postulaciones['se_postulo'] = 1

In [8]:
vistas.shape

(10471713, 25)

In [9]:
postulaciones.shape

(6188634, 25)

In [10]:
vistas_y_postulaciones = pd.concat([postulaciones, vistas], axis=0)
vistas_y_postulaciones.shape

(16660347, 25)

In [11]:
vistas_y_postulaciones.head()

,idaviso,esta_online,titulo,descripcion,contenido,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,...,dayofweek,hour,valor_educacion,sexo,edad,idnombre_area_pref,idnivel_laboral_pref,idtipo_de_trabajo_pref,idpostulante_int,se_postulo
0,1111556097,0,1120986480640,8230659354068575249,8159438729029635585,0,0,0,0,0,...,1,16,17,1,44,119,0,0,446521,1
1,1112370711,1,5629499534229504,4305833638884806680,4306765247469621377,0,0,0,0,0,...,3,14,17,1,44,119,0,0,446521,1
2,1112326655,1,16384,7886788350607107624,7959338346396781218,0,0,0,0,0,...,4,14,17,1,44,119,0,0,446521,1
3,1112350182,1,4611686018427404288,8861078463061660440,8789020298799968481,0,0,0,0,0,...,1,15,17,1,44,119,0,0,446521,1
4,1112383837,1,16392,6440571711296978984,6513419378897462402,0,0,0,0,0,...,2,14,17,1,44,119,0,0,446521,1


# Agrego la tercer clase

In [12]:
top100 = pd.read_csv("../test_final_100k.csv", encoding='utf-8')
top100.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [13]:
top100_full = pd.merge(top100, vistas_y_postulaciones, how='left', on=['idaviso', 'idpostulante'])
top100_full.head()

,id,idaviso,idpostulante,esta_online,titulo,descripcion,contenido,idzona,idciudad,idmapacalle,...,dayofweek,hour,valor_educacion,sexo,edad,idnombre_area_pref,idnivel_laboral_pref,idtipo_de_trabajo_pref,idpostulante_int,se_postulo
0,0,739260,6M9ZQR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,739260,6v1xdL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,739260,ezRKm9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,758580,1Q35ej,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,758580,EAN4J6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
excluded = top100_full[top100_full['esta_online'].isnull()]
excluded.shape

(60066, 26)

In [15]:
excluded = excluded[['id', 'idaviso', 'idpostulante']]
excluded.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [16]:
avisos_detalles = pd.read_csv('../data/adri_data/avisos_detalles.csv', encoding='utf-8')
avisos_detalles.head()

,idaviso,esta_online,titulo,descripcion,contenido,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa
0,1111556097,0,1120986480640,8230659354068575249,8159438729029635585,0,0,0,0,0,50,0
1,1112448447,1,576460769483300864,6228765605496356872,6157568899156279424,0,0,0,0,0,182,0
2,1112394600,0,1152921526081683456,7063905988445249617,6991840908279038473,1,0,520,0,0,34,0
3,1111266888,1,18014398511579136,8106761422736344128,8107605375203478536,0,0,0,0,2,142,0
4,1112426827,1,1125921381679104,8380078581765668944,8379514742535897097,0,0,0,0,0,114,0


In [17]:
excluded = pd.merge(excluded, avisos_detalles, how='inner', on='idaviso')
excluded.shape

(60066, 14)

In [18]:
postulantes = pd.read_csv('../data/adri_data/postulantes2.csv', encoding='utf-8')
postulantes.shape

(504407, 8)

In [19]:
excluded = pd.merge(excluded, postulantes, how='inner', on='idpostulante')
excluded.shape

(60066, 21)

In [20]:
excluded['se_postulo'] = -1

In [21]:
excluded['month'] = 0
excluded['day'] = 0
excluded['dayofweek'] = 0
excluded['hour'] = 0

excluded = excluded[vistas_y_postulaciones.columns]
excluded.head()

,idaviso,esta_online,titulo,descripcion,contenido,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,...,dayofweek,hour,valor_educacion,sexo,edad,idnombre_area_pref,idnivel_laboral_pref,idtipo_de_trabajo_pref,idpostulante_int,se_postulo
0,739260,0,292733975779082240,6066349110452061192,6066366341802117248,1,0,0,0,2,...,0,0,21,1,42,113,0,0,77465,-1
1,739260,0,292733975779082240,6066349110452061192,6066366341802117248,1,0,0,0,2,...,0,0,11,-1,30,14,0,0,94007,-1
2,739260,0,292733975779082240,6066349110452061192,6066366341802117248,1,0,0,0,2,...,0,0,12,1,36,152,0,0,395351,-1
3,758580,0,72057594037927936,0,0,1,0,0,0,3,...,0,0,17,-1,68,-1,-1,-1,17228,-1
4,758580,0,72057594037927936,0,0,1,0,0,0,3,...,0,0,10,1,32,173,0,0,155630,-1


In [22]:
# timestamps = pd.DataFrame(vistas_y_postulaciones.groupby('idaviso')['timestamp'].mean()).reset_index()
# timestamps.head()

In [23]:
# excluded = pd.merge(excluded, timestamps, on='idaviso', how='left')
# excluded.head()

In [24]:
# excluded = excluded[vistas_y_postulaciones.columns]
# excluded.head()

In [25]:
# timestampmean = vistas_y_postulaciones['timestamp'].mean()
# timestampmean

In [26]:
# excluded.loc[excluded['timestamp'].isnull(), 'timestamp'] = timestampmean
# excluded.head()

In [27]:
# excluded[excluded['timestamp'].isnull()].head()

In [28]:
# excluded.loc[excluded['timestamp'].isnull(), 'timestamp'] = timestampmean
# excluded.head()

# Mergeo con la tercer clase

In [29]:
vistas_y_postulaciones = pd.concat([vistas_y_postulaciones, excluded], axis=0)
vistas_y_postulaciones.head()

,idaviso,esta_online,titulo,descripcion,contenido,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,...,dayofweek,hour,valor_educacion,sexo,edad,idnombre_area_pref,idnivel_laboral_pref,idtipo_de_trabajo_pref,idpostulante_int,se_postulo
0,1111556097,0,1120986480640,8230659354068575249,8159438729029635585,0,0,0,0,0,...,1,16,17,1,44,119,0,0,446521,1
1,1112370711,1,5629499534229504,4305833638884806680,4306765247469621377,0,0,0,0,0,...,3,14,17,1,44,119,0,0,446521,1
2,1112326655,1,16384,7886788350607107624,7959338346396781218,0,0,0,0,0,...,4,14,17,1,44,119,0,0,446521,1
3,1112350182,1,4611686018427404288,8861078463061660440,8789020298799968481,0,0,0,0,0,...,1,15,17,1,44,119,0,0,446521,1
4,1112383837,1,16392,6440571711296978984,6513419378897462402,0,0,0,0,0,...,2,14,17,1,44,119,0,0,446521,1


In [30]:
# vistas_y_postulaciones.shape

In [31]:
# postulantes_ids = vistas_y_postulaciones[["idpostulante", "idpostulante2"]].drop_duplicates()
# postulantes_ids.to_csv('../data/adri_data/entrenamiento_id_postulantes.csv', encoding='utf-8', index=False)

In [32]:
# vistas_y_postulaciones.drop('idpostulante', axis=1, inplace=True)
# vistas_y_postulaciones.rename({'idpostulante_int': 'idpostulante'}, inplace=True, axis=1)
# vistas_y_postulaciones.head()

In [33]:
# vistas_y_postulaciones.to_csv('../data/adri_data/entrenamiento_postulaciones_vistas2.csv', encoding='utf-8', index=False)

# Luego de mergear

In [34]:
# vistas_y_postulaciones = pd.read_csv('../data/adri_data/entrenamiento_postulaciones_vistas2.csv', encoding='utf-8')

In [35]:
vistas_y_postulaciones.shape

(16720413, 25)

In [36]:
vistas_y_postulaciones.head()

,idaviso,esta_online,titulo,descripcion,contenido,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,...,dayofweek,hour,valor_educacion,sexo,edad,idnombre_area_pref,idnivel_laboral_pref,idtipo_de_trabajo_pref,idpostulante_int,se_postulo
0,1111556097,0,1120986480640,8230659354068575249,8159438729029635585,0,0,0,0,0,...,1,16,17,1,44,119,0,0,446521,1
1,1112370711,1,5629499534229504,4305833638884806680,4306765247469621377,0,0,0,0,0,...,3,14,17,1,44,119,0,0,446521,1
2,1112326655,1,16384,7886788350607107624,7959338346396781218,0,0,0,0,0,...,4,14,17,1,44,119,0,0,446521,1
3,1112350182,1,4611686018427404288,8861078463061660440,8789020298799968481,0,0,0,0,0,...,1,15,17,1,44,119,0,0,446521,1
4,1112383837,1,16392,6440571711296978984,6513419378897462402,0,0,0,0,0,...,2,14,17,1,44,119,0,0,446521,1


In [37]:
# dup = vistas_y_postulaciones.duplicated(['idaviso', 'idpostulante'])
# vistas_y_postulaciones[dup].shape

In [38]:
# vistas_y_postulaciones.drop_duplicates(['idaviso', 'idpostulante'], inplace=True)
# vistas_y_postulaciones.shape

In [39]:
vistas_y_postulaciones = vistas_y_postulaciones.sample(frac=1).reset_index(drop=True)

In [40]:
vistas_y_postulaciones.shape

(16720413, 25)

In [41]:
vistas_y_postulaciones.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16720413 entries, 0 to 16720412
Data columns (total 25 columns):
idaviso                   int64
esta_online               int64
titulo                    int64
descripcion               int64
contenido                 int64
idzona                    int64
idciudad                  int64
idmapacalle               int64
idtipo_de_trabajo         int64
idnivel_laboral           int64
idnombre_area             int64
iddenominacion_empresa    int64
idpostulante              object
month                     int64
day                       int64
dayofweek                 int64
hour                      int64
valor_educacion           int64
sexo                      int64
edad                      int64
idnombre_area_pref        int64
idnivel_laboral_pref      int64
idtipo_de_trabajo_pref    int64
idpostulante_int          int64
se_postulo                int64
dtypes: int64(24), object(1)
memory usage: 4.0 GB


In [45]:
vistas_y_postulaciones.drop(['idnombre_area_pref','idnivel_laboral_pref','idtipo_de_trabajo_pref'], inplace=True, axis=1)

In [46]:
vistas_y_postulaciones.head()

,idaviso,esta_online,titulo,descripcion,contenido,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,...,idpostulante,month,day,dayofweek,hour,valor_educacion,sexo,edad,idpostulante_int,se_postulo
0,1111644641,1,0,5476377147169832968,5476377146916601984,0,0,0,0,3,...,A3DK1pm,4,23,0,12,9,-1,37,131655,0
1,1112473531,0,2199023255552,4956212086242968337,4956211548969320545,1,0,402,0,0,...,4rPExVR,4,24,1,11,10,-1,29,49293,0
2,1112486431,0,141287785234432,9038406324004000984,9109603048669001881,0,0,0,0,1,...,RzJD5Z9,4,25,2,19,10,1,26,268513,0
3,1112377689,1,576460786663161856,32756925747167340,32281695856885898,0,0,0,0,0,...,JBrEOrr,4,21,5,5,8,-1,39,191184,0
4,1112465244,1,1152921504606846976,1297042761471492129,1224983978264494082,0,0,0,1,3,...,kPL9NlZ,4,20,4,20,12,1,26,405498,0


In [47]:
vistas_y_postulaciones.to_csv('../data/adri_data/entrenamiento_postulaciones_vistas18_full.csv', encoding='utf-8', index=False)

In [32]:
vistas_y_postulaciones['sexo'] = vistas_y_postulaciones['sexo'] * 100

In [33]:
vistas_y_postulaciones['sexo'] = vistas_y_postulaciones['sexo'] * 100
vistas_y_postulaciones['valor_educacion'] = vistas_y_postulaciones['valor_educacion'].apply(lambda x: 2**x)
vistas_y_postulaciones['month'] = vistas_y_postulaciones['month'].apply(lambda x: 2**x)
vistas_y_postulaciones['day'] = vistas_y_postulaciones['day'].apply(lambda x: 2**x)
vistas_y_postulaciones['dayofweek'] = vistas_y_postulaciones['dayofweek'].apply(lambda x: 2**x)
vistas_y_postulaciones['hour'] = vistas_y_postulaciones['hour'].apply(lambda x: 2**x)

In [28]:
vistas_y_postulaciones['month'] = vistas_y_postulaciones['month'].apply(lambda x: 2**x)
vistas_y_postulaciones['day'] = vistas_y_postulaciones['day'].apply(lambda x: 2**x)
vistas_y_postulaciones['dayofweek'] = vistas_y_postulaciones['dayofweek'].apply(lambda x: 2**x)
vistas_y_postulaciones['hour'] = vistas_y_postulaciones['hour'].apply(lambda x: 2**x)